In [360]:
import os 
import sys
import pygwalker as pyg
import pandas as pd

In [361]:

#Importing Configs
# Define the path where config.py is located
os.chdir('/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling')
config_file_path = os.getcwd()
print(config_file_path)

# Add this path to the sys.path
sys.path.append(config_file_path)

import config

#Configs
database_file = config.database
database_folder = config.database_folder
bert_models = config.bert_models
bert_models_local = config.bert_models_local
Sentiment_models = config.Sentiment_models

Body = config.Body
Model = config.Model
Model_Subfolder = f'/{Body} Texts/{Model}'
#Model_Folder = config.texts
Model_Folder = config.Sentiment_models
Model_Folder = Model_Folder + Model_Subfolder
Model_Folder = config.Sentiment_models

Body_2 = config.Body_2
Model_2 = config.Model_2
Model_Subfolder_2 = f'/{Body_2} Texts/{Model_2}'
Model_Folder_2 = config.texts
Model_Folder_2 = Model_Folder_2 + Model_Subfolder_2

/Users/kylenabors/Documents/GitHub/Finance-ML-Modeling


In [362]:
df = pd.read_csv(f"{Sentiment_models}/{Body}_{Model}_advanced_sentiment_texts.csv")
df_2 = pd.read_csv(f"{Sentiment_models}/{Body_2}_{Model_2}_advanced_sentiment_texts.csv")

In [363]:
dummies = pd.get_dummies(df['sentiment'], dtype=float)
df = df.join(dummies)
dummies_2 = pd.get_dummies(df_2['sentiment'], dtype=float)
df_2 = df_2.join(dummies)

In [364]:
print(df.head())
print(df_2.head())

   Unnamed: 0        date                                title  \
0           0  2023-08-31  Disinflation and the Phillips curve   
1           1  2023-08-31  Disinflation and the Phillips curve   
2           2  2023-08-31  Disinflation and the Phillips curve   
3           3  2023-08-31  Disinflation and the Phillips curve   
4           4  2023-08-31  Disinflation and the Phillips curve   

              type                                            segment  \
0  Isabel Schnabel  speech disinflation and the phillips curve spe...   
1  Isabel Schnabel  activity has moderated visibly , and forwardlo...   
2  Isabel Schnabel  but important pockets of resilience remain , e...   
3  Isabel Schnabel  headline inflation has come down , mainly on t...   
4  Isabel Schnabel  but underlying price pressures remain stubborn...   

  sentiment  confidence  neg  pos  
0       neg         0.6  1.0  0.0  
1       pos         1.0  0.0  1.0  
2       pos         1.0  0.0  1.0  
3       pos         

In [365]:
df_group = df.copy(deep=True)
df_group = df_group.drop(columns=['Unnamed: 0', 'title', 'type', 'segment', 'sentiment', 'confidence'])
df_group['monthly'] = df_group['date'].str[0:7]
df_group = df_group[['monthly', 'pos', 'neg']]
df_group = df_group.groupby('monthly', as_index = False).mean()
df_group['net'] = df_group['pos'] - df_group['neg']

print(df_2.head())

df_2_group = df_2.copy(deep=True)
df_2_group = df_2_group.drop(columns=['Unnamed: 0', 'title', 'type', 'segment', 'sentiment', 'confidence'])
df_2_group['date'] = df_2_group['date'].astype(str)
df_2_group['monthly'] = df_2_group['date'].str[0:7]
df_2_group = df_2_group[['monthly', 'pos', 'neg']]
df_2_group = df_2_group.groupby('monthly', as_index = False).mean()
df_2_group['net'] = df_2_group['pos'] - df_group['neg']

print(df_group.head())
print(df_2_group.head())

df_merged = df_group.merge(df_2_group, on='monthly', how='inner')
df_merged.fillna(0, inplace=True)

   Unnamed: 0        date                            title  \
0           0  19961219.0  Supervision of bank risk-taking   
1           1  19961219.0  Supervision of bank risk-taking   
2           2  19961219.0  Supervision of bank risk-taking   
3           3  19961219.0  Supervision of bank risk-taking   
4           4  19961219.0  Supervision of bank risk-taking   

                         type  \
0  Vice Chair Alice M. Rivlin   
1  Vice Chair Alice M. Rivlin   
2  Vice Chair Alice M. Rivlin   
3  Vice Chair Alice M. Rivlin   
4  Vice Chair Alice M. Rivlin   

                                             segment sentiment  confidence  \
0  i discovered when i joined the board of govern...       pos         0.8   
1  many of them said , enthusiastically , congrat...       pos         1.0   
2  then they asked with a bit of embarrassment , ...       pos         1.0   
3     or what will you find to do between meetings ?       pos         1.0   
4  the meetings they were aware of , o

In [366]:
print(df_merged.head())

df_merged.rename(columns={'pos_x': 'pos_1', 'neg_x': 'neg_1', 'net_x': 'net_1', 'pos_y': 'pos_2', 'neg_y': 'neg_2', 'net_y': 'net_2'}, inplace=True)

print(df_merged.head())

Empty DataFrame
Columns: [monthly, pos_x, neg_x, net_x, pos_y, neg_y, net_y]
Index: []
Empty DataFrame
Columns: [monthly, pos_1, neg_1, net_1, pos_2, neg_2, net_2]
Index: []


In [367]:
df_merged['net_diff'] = df_merged['net_1'] - df_merged['net_2']

In [368]:
funds = pd.read_excel('/Users/kylenabors/Documents/Database/Other Data/FedFundsRate.xlsx', sheet_name='Monthly')
print(funds.head())
energy = pd.read_csv('/Users/kylenabors/Documents/Database/Other Data/Energy Prices/US Energy CPI.csv')
print(energy.head())

        DATE  FEDFUNDS Date Adjusted
0 1999-12-31      5.45    2000-01-01
1 2000-01-31      5.73    2000-02-01
2 2000-02-29      5.85    2000-03-01
3 2000-03-31      6.02    2000-04-01
4 2000-04-30      6.27    2000-05-01
         DATE  CPIENGSL
0  2000-01-01     115.0
1  2000-02-01     118.8
2  2000-03-01     124.3
3  2000-04-01     120.9
4  2000-05-01     120.0


In [369]:
funds['Diff'] = funds['FEDFUNDS'].diff()


In [370]:
df_merged.to_csv(f"{Sentiment_models}/{Body}_{Model}_vs_{Body_2}_{Model_2}_advanced_sentiment_texts.csv", index=False)

In [371]:
gwalker = pyg.walk(df_merged)

Box(children=(HTML(value='<div id="ifr-pyg-22" style="height: auto">\n    <head>\n        <meta http-equiv="Co…